# Εργαστήριο 9
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Μνήμη εντολών
Στη συνέχεια του εργαστηρίου θα χρειαστούμε μια μνήμη μόνο για ανάγνωση (ROM) για να αποθηκεύουμε τις εντολές προς εκτέλεση. Στην Chisel αυτό μπορεί να γίνει με τη χρήση του `VecInit()` όπως στο επόμενο παράδειγμα:

~~~scala
val content = Vector(33.U,1256.U,555.U) 
val rom = VecInit(content)
~~~

Ο πιο πάνω κώδικας, όταν θα μεταφερθεί η σχεδίαση σε ένα πραγματικό κύκλωμα, θα δημιουργήσει ένα τμήμα υλικού που δέχεται μια είσοδο επιλογής (διεύθυνση) και παράγει την αντίστοιχη τιμή. Στο προηγούμενο παράδειγμα θα είναι rom(0)=33, rom(1)=1256 και rom(2)=555.

*Σημ.: Ανάλογα με την τεχνολογία κατασκευής, το κύκλωμα του VecInit θα υλοποιηθεί με ως πίνακας αναζήτησης (lookup table) ή με απλές πύλες.* 

### Γενικό module μνήμης εντολών
Στη συνέχεια δίνεται ένα *παραμετρικό* module `InstructionMemory` που περιέχει ένα τμήμα ROM για εντολές. Οι παράμετροι δημιουργίας είναι οι ακόλουθες:

* `addr_width`: εύρος διεύθυνσης (πόσα bits έχουν οι διευθύνσεις της μνήμης εντολών)

* `instr_width`: εύρος εντολών (πόσα bits έχει κάθε εντολή)

* `content`: ακολουθία με το περιεχόμενο της μνήμης εντολών (σε μορφή UInt)

Εκτελέστε το επόμενο κελί:

In [ ]:
class InstructionMemory(addr_width: Int, instr_width: Int, content: Seq[UInt]) extends Module {
  val io = IO(new Bundle {
    val address = Input(UInt(addr_width.W))
    val data_out = Output(UInt(instr_width.W))
  })
  val rom = VecInit(content)
    
  io.data_out := rom(io.address)
}

Δοκιμάστε το επόμενο κελί, όπου δημιουργούμε ένα InstructionMemory instance με εύρος διευθύνσεων 8 bits, εύρος εντολών 16 bits και αρχικοποίηση από ένα Vector με 3 τιμές και στη συνέχεια, για κάθε γραμμή της μνήμης ROM εισάγουμε τη διεύθυνσή της με poke και ελέγχουμε αν παίρνουμε το επιθυμητό αποτέλεσμα: 

In [ ]:
val instructions = Vector(33.U,1256.U,555.U)
test(new InstructionMemory(8,16,instructions)) { c =>
  for (i <- 0 until instructions.length) {
    c.io.address.poke(i.U)
    c.io.data_out.expect(instructions(i))
  }
}
println("SUCCESS!!")

## Άσκηση
Σκοπός της άσκησης είναι να κατασκευάσετε τη μονάδα
του program counter και της μνήμης εντολών της ΚΜΕ. Το module θα ονομάζεται `FetchUnit` και θα υλοποιεί
την εξής λειτουργικότητα:

* Το module θα είναι παραμετρικό ως προς το εύρος των διευθύνσεων (παράμετρος `addr_width`), το εύρος των εντολών (παράμετρος `instr_width`) και το περιεχόμενο της μνήμης εντολών (παράμετρος `content`).

* Θα περιέχει καταχωρητή `pcReg` με εύρος λέξης `addr_width` που θα κρατά την τιμή της διεύθυνσης της επόμενης εντολής. Με την έναρξη λειτουργίας ο καταχωρητής `pcReg` θα αρχικοποιείται στην τιμή 0.U

* Η έξοδος του καταχωρητή θα οδηγείται σε ένα `InstructionMemory` instance.

* Το module θα διαθέτει επίσης τις εξής εισόδους:

  * `branch_pc` (εύρος addr_width)
  * `pc_sel` (εύρος 1 bit)
  
  Σε κάθε κύκλο ρολογιού, η νέα τιμή του καταχωρητή `pcReg`θα ισούται:
  
  * Με την παλιά τιμή + 1, όταν το `pc_sel` είναι 0
  * Με το `branch_pc`, όταν το `pc_sel` είναι 1
  
* Παρατηρήστε τη χρήση του σήματος `addr_plus_1` για το σημείο του κυκλώματος μετά τον αθροιστή + 1
  
Υλοποιήστε το module `PcLogic` στο κελί που ακολουθεί σύμφωνα με το επόμενο σχήμα:

![fetchunit.png](https://mixstef.github.io/courses/comparch/labimg/fetchunit.png)

In [ ]:
class FetchUnit(addr_width: Int, instr_width: Int, content: Seq[UInt]) extends Module {
  val io = IO(new Bundle {
    val branch_pc = Input(UInt(addr_width.W))
    val pc_sel = Input(UInt(1.W))
    val next_pc = Output(UInt(addr_width.W))
    val instruction = Output(UInt(instr_width.W))
  })
  
  // instruction memory instance  
  val iMem = Module(new InstructionMemory(addr_width,instr_width,content))
  // pc register
  val pcReg = RegInit(0.U(addr_width.W))  
  // temp signal, output of +1 adder
  val addr_plus_1 = Wire(UInt())

  // συμπληρώστε τις συνδέσεις των σημάτων σύμφωνα με το σχήμα
    
}

Δοκιμάστε την ορθή λειτουργία του κυκλώματός σας με τον παρακάτω κώδικα:

In [ ]:
val instructions = Vector(33.U,1256.U,555.U)
test(new FetchUnit(8,16,instructions)) { c =>
  // test instr output when incrementing pc  
  c.io.pc_sel.poke(0.U)
  for (i <- 0 until instructions.length) {
    c.io.instruction.expect(instructions(i))
    c.clock.step()  
  }
  // test instr output with input branch pc  
  c.io.pc_sel.poke(1.U)
  c.io.branch_pc.poke(1.U)
  c.clock.step()
  c.io.instruction.expect(instructions(1))
}
println("SUCCESS!!")